# Tutorial 2.0: Imports, Paths, & Utils

This notebook contains all the Python imports required for Tutorial 2.

## 2.0.1 Python Imports

In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib agg

Please import:

In [1]:
import numpy
from IPython.display import HTML
from matplotlib import pyplot as plt
import itertools

from mbfit.utils.filesystem import Directory
from mbfit.utils.job_manager import SLURMJobManager, JobManager, LocalJobManager
from mbfit.calculator import QchemCalculator, OrcaCalculator

from mbfit.system.system_like import BondType
from mbfit.system.definition import BasicFragmentDefinition, BasicCompoundDefinition, AtomDefinition, BondDefinition
from mbfit.system.initializer import VSEPRInitializer
from mbfit.system.descriptor import fragment_definition_to_xyz, compound_definition_to_xyz, compound_to_xyz, compound_to_commented_xyz
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system import BasicCompound
from mbfit.system.transformer import SystemTransformer
from mbfit.system.copier import CompoundCopier
from mbfit.system.transformer import make_scan
from mbfit.system.region import BasicMutableRegionManager
from mbfit.visualization import render_system, render_systems, render_overlayed_systems
from mbfit.visualization import plot_scan
from mbfit.utils import constants

from mbfit.system.descriptor.energy_decomposition_comment_line_descriptor import EnergyDecompositionCommentLineDescriptor

from mbfit.calculator import calculate_energy, calculate_energies, minimize_structures, minimize_structure, calculate_vibrational_modes, write_vibrational_modes, read_vibrational_modes
from mbfit.system.io import write_fragment_definition, write_definition, write_system, write_systems, read_fragment_definition, read_definition, read_system, read_systems, write_system_and_energy, write_systems_and_energies, read_system_and_energy, read_systems_and_energies

from mbfit.parametrization import calculate_polarizabilities_by_XDM, calculate_radii_by_XDM, calculate_C6_by_XDM, calculate_d6_from_radii, get_ESP_grid_points, calculate_ESP_at_grid_points, fit_charges_to_ESP

from mbfit.configurations import NormalModesConfigurationGenerator

from mbfit.potential import calculate_Vphys_energies

## 2.0.2 Util Functions

In [74]:
qchem_job_template="""
module load cpu/0.15.4  gcc/10.2.0  mvapich2/2.3.6 qchem/5.4
conda activate p310
cd ~/software/development/MB-Fit_versions/QDMS_2025
export MBFIT_HOME=$PWD
source sourceme.sh
cd -

export QCSCRATCH=/expanse/lustre/scratch/$USER/temp_project/qscratch
export QCLOCALSCR=/scratch/$USER/job_$SLURM_JOB_ID

"""

def get_qchem_job_manager(job_scratch_directory: Directory) -> JobManager:
    job_scratch_directory.create()
    return LocalJobManager()
    return SLURMJobManager(
            input_path=f"{job_scratch_directory}/job.in",
            output_path=f"{job_scratch_directory}/job.out",
            log_path=f"{job_scratch_directory}/job.log",
            job_path=f"{job_scratch_directory}/job.job",
            job_output_path=f"{job_scratch_directory}/job.stdout",
            job_error_path=f"{job_scratch_directory}/job.stderr",
            max_queue_size=10,
            job_name="slurm_job_manager",
            allocation="csd888",
            reservation=None,
            partition="shared",
            qos=None,
            max_walltime_minutes=60,
            nodes=1,
            tasks=1,
            processors_per_task=32,
            template=qchem_job_template
    )

In [75]:
orca_job_template="""
module load cpu/0.17.3b  gcc/10.2.0/npcyll4  openmpi/4.1.1/ygduf2r
module load orca/5.0.4
export ORCA_SCRATCH=/scratch/$USER/job_$SLURM_JOB_ID
conda activate p310
cd ~/software/development/MB-Fit_versions/QDMS_2025
export MBFIT_HOME=$PWD
source sourceme.sh
cd -
"""

def get_orca_job_manager(job_scratch_directory: Directory) -> JobManager:
    job_scratch_directory.create()
    return LocalJobManager()
    return SLURMJobManager(
            input_path=f"{job_scratch_directory}/job.in",
            output_path=f"{job_scratch_directory}/job.out",
            log_path=f"{job_scratch_directory}/job.log",
            job_path=f"{job_scratch_directory}/job.job",
            job_output_path=f"{job_scratch_directory}/job.stdout",
            job_error_path=f"{job_scratch_directory}/job.stderr",
            max_queue_size=10,
            job_name="slurm_job_manager",
            allocation="csd888",
            reservation=None,
            partition="shared",
            qos=None,
            max_walltime_minutes=60,
            nodes=1,
            tasks=32,
            processors_per_task=1,
            template=orca_job_template
    )

## 2.0.3 File Paths

In this section, we we will setup the directory structure and file paths we will use in this notebook. Throughout this notebook, the following files will be created:

In [62]:
tutorial_2_workdir = Directory("Tutorial_2_workdir")
tutorial_2_workdir.create()

In [63]:
definitions_dir = tutorial_2_workdir.sub_directory("definitions")
definitions_dir.create()

In [64]:
fragment_CH3NH2_definition_path = definitions_dir.file("fragment_CH3NH2.def")
fragment_H2O_definition_path = definitions_dir.file("fragment_H2O.def")

In [65]:
monomer_CH3NH2_definition_path = definitions_dir.file("monomer_CH3NH2.def")
monomer_H2O_definition_path = definitions_dir.file("monomer_H2O.def")
dimer_CH3NH2_H2O_definition_path = definitions_dir.file("dimer_CH3NH2_H2O.def")
trimer_CH3NH2_H2O_H2O_definition_path = definitions_dir.file("trimer_CH3NH2_H2O_H2O.def")

In [66]:
structures_dir = tutorial_2_workdir.sub_directory("structures")
structures_dir.create()

In [67]:
monomer_CH3NH2_initial_structure_path = structures_dir.file("monomer_CH3NH2_initial_structure.xyz")
monomer_H2O_initial_structure_path = structures_dir.file("monomer_H2O_initial_structure.xyz")

monomer_CH3NH2_minimum_structure_path = structures_dir.file("monomer_CH3NH2_minimum_structure.xyz")
monomer_H2O_minimum_structure_path = structures_dir.file("monomer_H2O_minimum_structure.xyz")

In [68]:
scans_dir = tutorial_2_workdir.sub_directory("scans")
scans_dir.create()

In [69]:
scan_2b_1_path = scans_dir.file("scan_2b_1.xyz")
scan_2b_2_path = scans_dir.file("scan_2b_2.xyz")
scan_2b_3_path = scans_dir.file("scan_2b_3.xyz")

scan_3b_1_path = scans_dir.file("scan_3b_1.xyz")
scan_3b_2_path = scans_dir.file("scan_3b_2.xyz")
scan_3b_3_path = scans_dir.file("scan_3b_3.xyz")

In [70]:
scan_2b_energies_1_path = scans_dir.file("scan_2b_1.xyz")
scan_2b_energies_2_path = scans_dir.file("scan_2b_2.xyz")
scan_2b_energies_3_path = scans_dir.file("scan_2b_3.xyz")

scan_3b_energies_1_path = scans_dir.file("scan_3b_1.xyz")
scan_3b_energies_2_path = scans_dir.file("scan_3b_2.xyz")
scan_3b_energies_3_path = scans_dir.file("scan_3b_3.xyz")

In [71]:
logs_dir = tutorial_2_workdir.sub_directory("logs")
logs_dir.create()

In [72]:
jobs_dir = tutorial_2_workdir.sub_directory("jobs")
jobs_dir.create()

In [73]:
restart_dir = tutorial_2_workdir.sub_directory("restart")
restart_dir.create()

In [24]:
modes_dir = tutorial_2_workdir.sub_directory("modes")
modes_dir.create()

NameError: name 'tutorial_2_workdir' is not defined

In [ ]:
vibrational_modes_CH3NH2_path = modes_dir.file("CH3NH2.modes")

In [ ]:
configs_dir = tutorial_2_workdir.sub_directory("configs")
configs_dir.create()

In [ ]:
charge_configs_CH3NH2_path = configs_dir.file("CH3NH2_charge_parametrization_configs.xyz")

In [73]:
scratch_dir = tutorial_2_workdir.sub_directory("scratch")
scratch_dir.create()